# Forest Inference Library (FIL)
The forest inference library is used to load saved forest models of xgboost, lightgbm and perform inference on them. It can be used to perform both classification and regression. In this notebook, we'll begin by fitting a model with XGBoost and saving it. We'll then load the saved model into FIL and use it to infer on new data.

FIL works in the same way with lightgbm model as well.

The model accepts both numpy arrays and cuDF dataframes. In order to convert your dataset to cudf format please read the cudf documentation on https://docs.rapids.ai/api/cudf/stable. 

For additional information on the forest inference library please refer to the documentation on https://docs.rapids.ai/api/cuml/stable/api.html#forest-inferencing

In [2]:
%load_ext memory_profiler

In [3]:
%%memit
import numpy as np
import os

from cuml.test.utils import array_equal
from cuml.common.import_utils import has_xgboost

from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
    
from cuml import ForestInference

peak memory: 338.24 MiB, increment: 288.43 MiB


### Check for xgboost
Checks if xgboost is present, if not then it throws an error.

In [4]:
%%memit
if has_xgboost():
    import xgboost as xgb
else:
    raise ImportError("Please install xgboost using the conda package,"
                      "e.g.: conda install -c conda-forge xgboost")

peak memory: 344.41 MiB, increment: 5.50 MiB


## Define parameters

In [5]:
%%memit
# synthetic data size
n_rows = 2000000
n_columns = 100
n_categories = 2
random_state = np.random.RandomState(43210)

# fraction of data used for model training
train_size = 0.5

# trained model output filename
model_path = 'xgb_2000.model'

peak memory: 344.60 MiB, increment: 0.01 MiB


## Generate data

In [5]:
%%memit
# create the dataset
X, y = make_classification(
    n_samples=n_rows,
    n_features=n_columns,
    n_informative=int(n_columns/5),
    n_classes=n_categories,
    random_state=42
)

# convert the dataset to float32
X = X.astype('float32')
y = y.astype('float32')

# split the dataset into training and validation splits
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=train_size)

peak memory: 5144.52 MiB, increment: 4800.50 MiB


## Load Forest Inference Library (FIL)

The load function of the ForestInference class accepts the following parameters:

       filename : str
           Path to saved model file in a treelite-compatible format
           (See https://treelite.readthedocs.io/en/latest/treelite-api.html
        output_class : bool
           If true, return a 1 or 0 depending on whether the raw prediction
           exceeds the threshold. If False, just return the raw prediction.
        threshold : float
           Cutoff value above which a prediction is set to 1.0
           Only used if the model is classification and output_class is True
        algo : string name of the algo from (from algo_t enum)
             'NAIVE' - simple inference using shared memory
             'TREE_REORG' - similar to naive but trees rearranged to be more
                              coalescing-friendly
             'BATCH_TREE_REORG' - similar to TREE_REORG but predicting
                                    multiple rows per thread block
        model_type : str
            Format of saved treelite model to load.
            Can be 'xgboost', 'lightgbm'

## Loaded the saved model
Use FIL to load the saved xgboost model

In [6]:
%%time
%%memit
fil_model_naive = ForestInference.load(
    filename=model_path,
    algo='NAIVE',
    output_class=True,
    threshold=0.50,
    model_type='xgboost'
)

peak memory: 1859.49 MiB, increment: 1514.89 MiB
CPU times: user 1.22 s, sys: 631 ms, total: 1.85 s
Wall time: 1.81 s


In [7]:
%%time
%%memit
fil_model_btr = ForestInference.load(
    filename=model_path,
    algo='BATCH_TREE_REORG',
    output_class=True,
    threshold=0.50,
    model_type='xgboost'
)

peak memory: 3859.42 MiB, increment: 1999.91 MiB
CPU times: user 2.35 s, sys: 904 ms, total: 3.25 s
Wall time: 3.29 s


## Predict using FIL

In [8]:
%%time
# perform prediction on the model loaded from path
fil_preds_naive = fil_model_naive.predict(X_validation)

CPU times: user 320 ms, sys: 213 ms, total: 532 ms
Wall time: 531 ms


In [9]:
%%time
# perform prediction on the model loaded from path
fil_preds_btr = fil_model_btr.predict(X_validation)

CPU times: user 219 ms, sys: 97 µs, total: 219 ms
Wall time: 218 ms


In [10]:
len(fil_preds_naive)

1000000